In [1]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [2]:
df = pd.read_csv("sample_text.csv")
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Create source embeddings for the text column

In [3]:
from sentence_transformers import SentenceTransformer

d:\Samuel_Learnings\New folder\sam_vs\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

d:\Samuel_Learnings\New folder\sam_vs\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\Samuel_Learnings\New folder\sam_vs\lib\site-packages\transformers\tokenization_utils

In [5]:
vectors.shape

(8, 768)

In [6]:
dim = vectors.shape[1]
dim

768

### Build a FAISS Index for vectors

In [7]:
import faiss          # faiss - Facebook AI Similerity Search

index = faiss.IndexFlatL2(dim)     # It computes L2 Eucledian Distance between vectors
                                   #The 'dim' parameter represents the dimensionality of the vectors you're working with.

In [8]:
index.add(vectors)

### Step 4 : Encode search text using same encorder and normalize the output vector


In [9]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [13]:
type(vec)

numpy.ndarray

In [14]:
import numpy as np
svec = vec.reshape(1,-1)  # here 1 repesents number of row
                          # -1 repesents auto calculation of number of columns based on number of elements in given input array.
svec.shape

(1, 768)

### Search for similar vector in the FAISS index created


In [19]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844835, 1.4039093]], dtype=float32)

In [20]:
I

array([[3, 2]], dtype=int64)

In [21]:
I.tolist()

[[3, 2]]

In [22]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [23]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [ ]:
search_query